In [1]:
from typing import Dict, Optional, Tuple
from sympy import Ci
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torchvision.datasets import Flowers102
from torchvision import transforms
from torchvision.utils import save_image, make_grid

from mindiffusion.unet import NaiveUnet
from mindiffusion.ddpm import DDPM

# список изменений в оригинальном коде:
1. датасет взят из torchvision.datasets.Flowers102
2. transforms.Resize([64,64]) - добавлен ресайз для трансформации данных, также поворот по вертикали/горизонтали 
3. ddpm.sample - изменен размер -> (3, 64, 64)
4. незначительные изменения из-за работы на винде на локальной машине: батч сайз уменьшен до 16, num_workers=0 в силу особенностей pytorch на windows, изменение различных Path

In [14]:
def train_Flowers102(
    n_epoch: int = 100, device: str = "cuda:0", load_pth: Optional[str] = None #  номер куды на 0
) -> None:

    ddpm = DDPM(eps_model=NaiveUnet(3, 3, n_feat=128), betas=(1e-4, 0.02), n_T=1000) # 3 входных канала, 3 выходных, 128 фич

    if load_pth is not None:
        ddpm.load_state_dict(torch.load("ddpm_Flowers.pth"))

    ddpm.to(device)

    tf = transforms.Compose(
        [ transforms.ToTensor(), transforms.Resize([64,64]), transforms.RandomVerticalFlip(0.1),transforms.RandomHorizontalFlip(0.1) ] ) # добален ресайз и повороты

    dataset = Flowers102(
        "./data",
        download=True,
        transform=tf,
    )

    dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0) # батч сайз уменьшил и число работников 0, потому что я на винде работаю
    optim = torch.optim.Adam(ddpm.parameters(), lr=1e-5)

    for i in range(n_epoch):
        print(f"Epoch {i} : ")
        
        ddpm.train()

        pbar = tqdm(dataloader)
        loss_ema = None
        for x, _ in pbar:
            ### break ###
            optim.zero_grad()
            x = x.to(device)
            loss = ddpm(x)
            loss.backward()
            if loss_ema is None:
                loss_ema = loss.item()
            else:
                loss_ema = 0.9 * loss_ema + 0.1 * loss.item()
            pbar.set_description(f"loss: {loss_ema:.4f}")
            optim.step()

        ddpm.eval()
        with torch.no_grad():
            xh = ddpm.sample(8, (3, 64, 64), device) # здесь размер 3*64*64 у торча C*H*W 
            xset = torch.cat([xh, x[:8]], dim=0)
            grid = make_grid(xset, normalize=True, value_range=(-1, 1), nrow=4) 
            save_image(grid, f"./contents/ddpm_sample_Flowers{i}.png") # изменил название

            # save model
            torch.save(ddpm.state_dict(), f"./ddpm_Flowers.pth") # название сохраненных весов изменено





In [15]:
train_Flowers102()

Epoch 0 : 


loss: 0.7768: 100%|██████████| 64/64 [00:20<00:00,  3.19it/s]


Epoch 1 : 


loss: 0.4520: 100%|██████████| 64/64 [00:19<00:00,  3.21it/s]


Epoch 2 : 


...

Epoch 10 : 


loss: 0.0951: 100%|██████████| 64/64 [00:20<00:00,  3.08it/s]


Epoch 11 : 


loss: 0.0911:  55%|█████▍    | 35/64 [00:12<00:09,  2.91it/s]


KeyboardInterrupt: 